<a href="https://colab.research.google.com/github/ankesh86/LLMProjects/blob/main/MCQAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install langchain-groq langchain


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.7/116.7 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.

In [34]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from google.colab import userdata

# Set up additional parameters if possible
model_parameters = {
    #"max_tokens": 1024,  # Setting a large enough limit to handle verbose medical questions
    "temperature": 0.00001,  # Keeping the output deterministic
    #"topP": 0.7,
}

model = ChatGroq(**model_parameters, groq_api_key=userdata.get('GROQ_API_KEY'), model_name="mixtral-8x7b-32768")
#llama3-70b-8192")
#"mixtral-8x7b-32768")

In [35]:
from typing import List

from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from pydantic import BaseModel, Field

class MedicalMCQ(BaseModel):
    question: str = Field(description="The medical multiple-choice question.")
    options: List[str] = Field(description="List of answer options for the MCQ.")
    correct_option: int = Field(description="Index of the correct answer option, zero-indexed.")
    explanation: str = Field(description="Explanation or additional context related to the question.")
    reasoning_abilities: List[str] = Field(description="Reasoning abilities: Recall-Based Questions, Application-Based Questions, Analytical Reasoning, Problem-Solving, Critical Thinking, Clinical Decision-Making.")
    difficulty_level: str = Field(description="Difficulty levels: Hard, Moderate, Easy.")

# Initialize the parser for the MedicalMCQ model
mcq_parser = PydanticOutputParser(pydantic_object=MedicalMCQ)

# Create a prompt template for generating a structured output from unformatted text
mcq_prompt = PromptTemplate(
    template="""
    Considering you are an analyst and have to categorize the question according to the section in which the question lies most.
    Extract the medical MCQ where there would be only and only 4 option from the following text and analyze the following MCQ.
    A. If Explanation is missing, provide an explanation else, give the explanation provided.
    B. Consider the following difficulty levels on the in terms of the cognitive processes involved and the knowledge required to answer them correctly: Hard, Moderate, Easy.
    C. Consider the following reasoning abilities: Recall-Based Questions, Application-Based Questions, Analytical Reasoning, Problem-Solving, Critical Thinking, Clinical Decision-Making.
    Assess various types of reasoning abilities:
    Recall-Based Questions:
    Description: These questions test the candidate’s ability to recall facts, figures, and concepts learned during their medical education.
    Example: What is the normal range of serum creatinine in adults?
    Application-Based Questions:
    Description: These questions require candidates to apply their knowledge to clinical scenarios.
    Example: A patient presents with symptoms of polyuria and polydipsia. Which of the following tests would confirm the diagnosis of diabetes insipidus?
    Analytical Reasoning:
    Description: This involves interpreting data from graphs, studies, or experiments to make clinical decisions.
    Example: Reviewing a set of lab results and determining the most likely diagnosis or the best next step in management.
    Problem-Solving:
    Description: These questions test the ability to solve complex problems, often involving multiple steps or integrating knowledge from multiple areas.
    Example: A patient with a history of renal failure and hypertension presents with a severe headache and blurred vision. What should be the immediate course of action?
    Critical Thinking:
    Description: Questions that assess the ability to critically evaluate information, assess evidence, and make judgments that help in patient care.
    Example: Evaluating the risks and benefits of a surgical procedure in a high-risk patient.
    Clinical Decision-Making:
    Description: These involve scenarios where candidates must choose the best clinical management strategy based on patient history, examination findings, and diagnostic results.
    Example: Choosing the appropriate antibiotic therapy based on culture and sensitivity reports.

     \n{format_instructions}\n{query}\n""",
    input_variables=["query"],  # This will contain the unformatted MCQ text
    partial_variables={"format_instructions": mcq_parser.get_format_instructions()},
)

# Assuming 'model' refers to a suitable model from LangChain that can parse the text to JSON
chain = mcq_prompt | model | mcq_parser


In [37]:
# Example text input
mcq_text = """
QuesNo	Question	OptionA	OptionB	OptionC	OptionD	Answer	Explanation	Subject	Tag	EnteredDate
314	Which of the following best describes a posthumous child?
	Delivery of a macerated fetus
	Child is delivered after death of biological father
	Child is born to an unmarried couple
	Child is abandoned by the parents
	B
	A posthumous child is a child delivered after the death of the biological father, or a child delivered after the death of the biological mother, usually when delivered by a cesarean section.
Illegitimate child is a child born out of wedlock
i.e. when the parents are not married.
Supposititious child or fictitious child is when a woman pretends to go through pregnancy and deliverv. Later, she oroduces a livina child as her
	NULL	CamScanner 10-29-2022 01.44	2022-10-29 14:15:14.95
"""

# Invoke the chain with the unformatted text as input
extracted_mcq = chain.invoke({"query": mcq_text})

# Access the extracted data
print(f"Question: {extracted_mcq.question}")
print(f"Options: {extracted_mcq.options}")
print(f"Correct Option: {extracted_mcq.correct_option}")
print(f"Explanation: {extracted_mcq.explanation}")
print(f"Reasoning ability: {extracted_mcq.reasoning_abilities}")
print(f"Difficulty level: {extracted_mcq.difficulty_level}")

Question: Which of the following best describes a posthumous child?
Options: ['Delivery of a macerated fetus', 'Child is delivered after death of biological father', 'Child is born to an unmarried couple', 'Child is abandoned by the parents']
Correct Option: 1
Explanation: A posthumous child is a child delivered after the death of the biological father, or a child delivered after the death of the biological mother, usually when delivered by a cesarean section.
Reasoning ability: ['Recall-Based Questions']
Difficulty level: Easy
